In [1]:
import os

In [2]:
!pwd
os.chdir("../")
!pwd

/home/tensorthiru/ml_lifecycle_sample/research
/home/tensorthiru/ml_lifecycle_sample


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidtionConfig:
    root_dir: Path
    data_file: Path
    status_file: Path
    all_schema: dict

In [9]:
##Schema
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/wine-quality.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [5]:
from src.pipeline_sample.utils.common import *
from src.pipeline_sample.constants import *

In [27]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 schema_file_path: Path = SCHEMA_FILE_PATH,
                 params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        if type(self.config.artifacts_root) == str:
            create_directories([self.config.artifacts_root])
        else:
            create_directories(self.config.artifacts_root)
    
    def get_data_validation_config(self) -> DataValidtionConfig:
        
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        data_validation_config = DataValidtionConfig(
            root_dir=Path(config.root_dir),
            data_file=Path(config.data_file),
            status_file=Path(config.status_file),
            all_schema = schema
        )
        
        return data_validation_config

In [29]:
import os
from src.pipeline_sample import logger

In [38]:
##Component
import json

class DataValidation:
    def __init__(self, config: DataValidtionConfig):
        self.config = config
        create_directories([self.config.root_dir])
    
    def initiate_data_validation(self):
        
        ##Read the data file
        data = pd.read_csv(self.config.data_file)
        all_cols = data.columns.tolist()
        
        ##Get schema
        schema = self.config.all_schema
        
        for col in all_cols:
            if col not in schema:
                status = {"validation_status": "Failed", "reason": f"Column {col} not in schema"}
                with open(self.config.status_file, "w") as f:
                    json.dump(status, f)
            
            else:
                status = {"validation_status": "Success"}
                with open(self.config.status_file, "w") as f:
                    json.dump(status, f)
        
        

In [39]:
cfg = ConfigurationManager().get_data_validation_config()

[2025-01-30 13:27:51,805: INFO: common: YAML file laoded successfully: config/config.yaml]
[2025-01-30 13:27:51,806: INFO: common: YAML file laoded successfully: params.yaml]
[2025-01-30 13:27:51,809: INFO: common: YAML file laoded successfully: schema.yaml]
[2025-01-30 13:27:51,810: INFO: common: Directory created: artifacts]


In [40]:
data_validation = DataValidation(cfg)
data_validation.initiate_data_validation()

[2025-01-30 13:27:53,954: INFO: common: Directory created: artifacts/data_validation]
